# Preview the data

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from ucimlrepo import fetch_ucirepo 

In [2]:
statlog_german_credit_data = fetch_ucirepo(id=144) 
  
X = statlog_german_credit_data.data.features
y = statlog_german_credit_data.data.targets

In [3]:
X

,Attribute1,Attribute2,Attribute3,Attribute4,Attribute5,Attribute6,Attribute7,Attribute8,Attribute9,Attribute10,Attribute11,Attribute12,Attribute13,Attribute14,Attribute15,Attribute16,Attribute17,Attribute18,Attribute19,Attribute20
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,4,A121,67,A143,A152,2,A173,1,A192,A201
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,2,A121,22,A143,A152,1,A173,1,A191,A201
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,3,A121,49,A143,A152,1,A172,2,A191,A201
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,4,A122,45,A143,A153,1,A173,2,A191,A201
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,4,A124,53,A143,A153,2,A173,2,A191,A201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,A14,12,A32,A42,1736,A61,A74,3,A92,A101,4,A121,31,A143,A152,1,A172,1,A191,A201
996,A11,30,A32,A41,3857,A61,A73,4,A91,A101,4,A122,40,A143,A152,1,A174,1,A192,A201
997,A14,12,A32,A43,804,A61,A75,4,A93,A101,4,A123,38,A143,A152,1,A173,1,A191,A201
998,A11,45,A32,A43,1845,A61,A73,4,A93,A101,4,A124,23,A143,A153,1,A173,1,A192,A201


In [4]:
y

,class
0,1
1,2
2,1
3,1
4,2
...,...
995,1
996,1
997,1
998,2


# Main Orchestraion Code

In [5]:
import ray
import mlflow
from preprocess import load_data
from models import get_models
from train import train_model
from sklearn.model_selection import train_test_split

In [ ]:
!mlflow ui

In [8]:
ray.init()

2025-05-27 23:53:50,599	INFO worker.py:1888 -- Started a local Ray instance.


Python version:,3.12.4
Ray version:,2.46.0


In [6]:
X, y = load_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

c:\Users\Kacper\Documents\lsc_proj\preprocess.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = LabelEncoder().fit_transform(X[col])
c:\Users\Kacper\Documents\lsc_proj\preprocess.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = LabelEncoder().fit_transform(X[col])
c:\Users\Kacper\Documents\lsc_proj\preprocess.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [7]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("CreditRisk_Parallel")

models = get_models()
futures = []

for name, model in models.items():
    future = train_model.remote(name, model, X_train, X_test, y_train, y_test)
    futures.append(future)

results = ray.get(futures)

2025-05-28 00:34:10,060	INFO worker.py:1888 -- Started a local Ray instance.
(train_model pid=2256) c:\Users\Kacper\Documents\lsc_proj\venv\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
(train_model pid=2256)   y = column_or_1d(y, warn=True)
(train_model pid=2360) c:\Users\Kacper\Documents\lsc_proj\venv\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
(train_model pid=2360)   return fit_method(estimator, *args, **kwargs)
(train_model pid=2256) c:\Users\Kacper\Documents\lsc_proj\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
(train_model pid=2256) STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.
(train_model pid=2256)

(train_model pid=2256) 🏃 View run LogisticRegression at: http://127.0.0.1:5000/#/experiments/734475923313736425/runs/1524c85e417b4bf4b3372b55229c0865
(train_model pid=2256) 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/734475923313736425
(train_model pid=14832) 🏃 View run SVM at: http://127.0.0.1:5000/#/experiments/734475923313736425/runs/324ac101e12848eeac7e247989157977


(train_model pid=2360) 🏃 View run RandomForest at: http://127.0.0.1:5000/#/experiments/734475923313736425/runs/38e2f907a86845a8b47d3c225fbc7843
(train_model pid=2360) 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/734475923313736425


In [8]:
print("\nTraining Summary:")
for name, acc, f1 in results:
    print(f"{name} -> Accuracy: {acc:.2f}, F1-score: {f1:.2f}")


Training Summary:
LogisticRegression -> Accuracy: 0.77, F1-score: 0.84
RandomForest -> Accuracy: 0.77, F1-score: 0.85
SVM -> Accuracy: 0.71, F1-score: 0.83
